In [82]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import itertools
import scipy.stats as st
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
import pingouin as pg
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import mutual_info_classif
import random
from random import randint



# Import data and choose which (if any) checks we are removing

In [32]:
def load_data(genpop_or_enriched, remove_checks):
    assert(genpop_or_enriched in ['genpop', 'enriched'])
    assert(remove_checks in ['no', 'gridonly', 'all'])
    if genpop_or_enriched == 'genpop':
        if remove_checks == 'no':
            data_path = '../data/mydata_1general_forCORR_removedNOchecks.csv'
        elif remove_checks == 'gridonly':
            data_path = '../data/mydata_1general_forCORR_removedGRIDchecks.csv'
        elif remove_checks == 'all':
            data_path = '../data/mydata_1general_forCORR_removedchecks.csv'
    elif genpop_or_enriched == 'enriched':
        if remove_checks == 'no':
            data_path = '../data/mydata_1highrisk_forCORR_removedNOchecks.csv'
        elif remove_checks == 'gridonly':
            data_path = '../data/mydata_1highrisk_forCORR_removedGRIDchecks.csv'
        elif remove_checks == 'all':
            data_path = '../data/mydata_1highrisk_forCORR_removedchecks.csv'  
    mydata = pd.read_csv(data_path)
    mydata = mydata.rename(columns={'Unnamed: 0': 'Subject'}) 
    return(mydata)

In [42]:
# loading both samples, then combining

mydata_genpop = load_data(genpop_or_enriched = 'genpop', remove_checks = 'gridonly')
mydata_enriched = load_data(genpop_or_enriched = 'enriched', remove_checks = 'gridonly')

mydata = pd.concat([mydata_genpop, mydata_enriched])
#mydata = mydata.rename(columns={'Unnamed: 0': 'Subject'})
mydata.head()

,Subject,hitop_sum,hitop_sum_recontact,baars_sum,baars_sum_recontact,phq_sum,phq_sum_recontact,gad_sum,gad_sum_recontact,mood_bothered,mood_bothered_recontact,anxiety_bothered,anxiety_bothered_recontact,attention_bothered,attention_bothered_recontact,moodanxiety_bothered,moodanxiety_bothered_recontact,baars_inattention_sum,baars_inattention_sum_recontact,baars_hyperactivity_sum,baars_hyperactivity_sum_recontact,baars_impulsivity_sum,baars_impulsivity_sum_recontact,baars_sct_sum,baars_sct_sum_recontact,hitop_anhedonic_depression,hitop_anhedonic_depression_recontact,hitop_anxious_worry,hitop_anxious_worry_recontact,hitop_appetite_gain,hitop_appetite_gain_recontact,hitop_appetite_loss,hitop_appetite_loss_recontact,hitop_cognitive_problems,hitop_cognitive_problems_recontact,hitop_hyposomnia,hitop_hyposomnia_recontact,hitop_indecisiveness,hitop_indecisiveness_recontact,hitop_insomnia,hitop_insomnia_recontact,hitop_panic,hitop_panic_recontact,hitop_separation_insecurity,hitop_separation_insecurity_recontact,hitop_shame_guilt,hitop_shame_guilt_recontact,hitop_situational_phobia,hitop_situational_phobia_recontact,hitop_social_anxiety,hitop_social_anxiety_recontact,hitop_well_being,hitop_well_being_recontact,hitop157,hitop157_recontact,hitop81,hitop81_recontact,hitop34,hitop34_recontact,hitop54,hitop54_recontact,hitop243,hitop243_recontact,hitop182,hitop182_recontact,hitop69,hitop69_recontact,hitop89,hitop89_recontact,hitop50,hitop50_recontact,hitop129,hitop129_recontact,hitop265,hitop265_recontact,hitop124,hitop124_recontact,hitop231,hitop231_recontact,hitop93,hitop93_recontact,hitop67,hitop67_recontact,hitop245,hitop245_recontact,hitop281,hitop281_recontact,hitop141,hitop141_recontact,hitop40,hitop40_recontact,hitop204,hitop204_recontact,hitop21,hitop21_recontact,hitop236,hitop236_recontact,hitop280,hitop280_recontact,hitop84,hitop84_recontact,hitop120,hitop120_recontact,hitop77,hitop77_recontact,hitop92,hitop92_recontact,hitop258,hitop258_recontact,hitop39,hitop39_recontact,hitop254,hitop254_recontact,hitop215,hitop215_recontact,hitop95,hitop95_recontact,hitop106,hitop106_recontact,hitop283,hitop283_recontact,hitop16,hitop16_recontact,hitop20,hitop20_recontact,hitop189,hitop189_recontact,hitop1,hitop1_recontact,hitop136,hitop136_recontact,hitop246,hitop246_recontact,hitop248,hitop248_recontact,hitop257,hitop257_recontact,hitop114,hitop114_recontact,hitop117,hitop117_recontact,hitop250,hitop250_recontact,hitop200,hitop200_recontact,hitop160,hitop160_recontact,hitop23,hitop23_recontact,hitop165,hitop165_recontact,hitop244,hitop244_recontact,hitop9,hitop9_recontact,hitop142,hitop142_recontact,hitop230,hitop230_recontact,hitop149,hitop149_recontact,hitop247,hitop247_recontact,hitop99,hitop99_recontact,hitop66,hitop66_recontact,hitop240,hitop240_recontact,hitop222,hitop222_recontact,hitop90,hitop90_recontact,hitop113,hitop113_recontact,hitop278,hitop278_recontact,hitop203,hitop203_recontact,hitop159,hitop159_recontact,hitop123,hitop123_recontact,hitop275,hitop275_recontact,hitop268,hitop268_recontact,hitop225,hitop225_recontact,hitop143,hitop143_recontact,hitop151,hitop151_recontact,hitop181,hitop181_recontact,hitop211,hitop211_recontact,hitop17,hitop17_recontact,hitop126,hitop126_recontact,hitop5,hitop5_recontact,hitop261,hitop261_recontact,hitop220,hitop220_recontact,hitop15,hitop15_recontact,hitop72,hitop72_recontact,hitop140,hitop140_recontact,hitop109,hitop109_recontact,hitop197,hitop197_recontact,hitop104,hitop104_recontact,todayhitop157,todayhitop157_recontact,todayhitop81,todayhitop81_recontact,todayhitop34,todayhitop34_recontact,todayhitop54,todayhitop54_recontact,todayhitop243,todayhitop243_recontact,todayhitop182,todayhitop182_recontact,todayhitop69,todayhitop69_recontact,todayhitop89,todayhitop89_recontact,todayhitop50,todayhitop50_recontact,todayhitop129,todayhitop129_recontact,todayhitop265,todayhitop265_recontact,todayhitop124,todayhitop124_recontact,todayhitop231,todayhitop231_recontact,todayhitop93,todayhitop93_recontact

In [43]:
for c in mydata.columns:
    print(c)

Subject
hitop_sum
hitop_sum_recontact
baars_sum
baars_sum_recontact
phq_sum
phq_sum_recontact
gad_sum
gad_sum_recontact
mood_bothered
mood_bothered_recontact
anxiety_bothered
anxiety_bothered_recontact
attention_bothered
attention_bothered_recontact
moodanxiety_bothered
moodanxiety_bothered_recontact
baars_inattention_sum
baars_inattention_sum_recontact
baars_hyperactivity_sum
baars_hyperactivity_sum_recontact
baars_impulsivity_sum
baars_impulsivity_sum_recontact
baars_sct_sum
baars_sct_sum_recontact
hitop_anhedonic_depression
hitop_anhedonic_depression_recontact
hitop_anxious_worry
hitop_anxious_worry_recontact
hitop_appetite_gain
hitop_appetite_gain_recontact
hitop_appetite_loss
hitop_appetite_loss_recontact
hitop_cognitive_problems
hitop_cognitive_problems_recontact
hitop_hyposomnia
hitop_hyposomnia_recontact
hitop_indecisiveness
hitop_indecisiveness_recontact
hitop_insomnia
hitop_insomnia_recontact
hitop_panic
hitop_panic_recontact
hitop_separation_insecurity
hitop_separation_insec

# Replace the OG hitop measures with invariant cores

In [44]:
mydata['hitop_anhedonic_depression_invcore'] = mydata['hitop39'] + mydata['hitop77'] + mydata['hitop84'] +\
mydata['hitop93'] + mydata['hitop123'] + mydata['hitop182'] + mydata['hitop230'] + mydata['hitop246']

mydata['hitop_anxious_worry_invcore'] = mydata['hitop20'] + mydata['hitop34'] + mydata['hitop89'] +\
mydata['hitop203'] + mydata['hitop248'] 

mydata['hitop_social_anxiety_invcore'] = mydata['hitop1'] + mydata['hitop17'] + mydata['hitop114'] +\
mydata['hitop124'] + mydata['hitop129'] + mydata['hitop204'] + mydata['hitop222'] + mydata['hitop258']

In [45]:
## RUN FOR GENPOP ONLY
#mydata['hitop_anxious_worry_invcore'] = mydata['hitop20'] + mydata['hitop34'] + mydata['hitop89'] + mydata['hitop203'] + mydata['hitop248']
#mydata['hitop_appetite_gain_invcore'] = mydata['hitop141'] + mydata['hitop243'] + mydata['hitop275']
#mydata['hitop_separation_insecurity_invcore'] = mydata['hitop40'] + mydata['hitop50'] + mydata['hitop69'] + mydata['hitop81'] + mydata['hitop136'] + mydata['hitop151'] + mydata['hitop197']
#mydata['hitop_social_anxiety_invcore'] = mydata['hitop17'] + mydata['hitop114'] + mydata['hitop117'] + mydata['hitop124'] + mydata['hitop129'] + mydata['hitop204'] + mydata['hitop222']
#mydata['hitop_well_being_invcore'] = mydata['hitop9'] + mydata['hitop23'] + mydata['hitop106'] + mydata['hitop149'] + mydata['hitop200'] + mydata['hitop244'] + mydata['hitop250'] + mydata['hitop281']

In [46]:
## RUN FOR ENRICHED ONLY
#mydata['hitop_anhedonic_depression_invcore'] = mydata['hitop39'] + mydata['hitop77'] + mydata['hitop84'] + mydata['hitop93'] + mydata['hitop123'] + mydata['hitop182'] + mydata['hitop230'] + mydata['hitop246']
#mydata['hitop_separation_insecurity_invcore'] = mydata['hitop40'] + mydata['hitop50'] + mydata['hitop69'] + mydata['hitop81'] + mydata['hitop113'] + mydata['hitop136'] + mydata['hitop151'] + mydata['hitop197']
#mydata['hitop_social_anxiety_invcore'] = mydata['hitop114'] + mydata['hitop117'] + mydata['hitop124'] + mydata['hitop129'] + mydata['hitop204'] + mydata['hitop222'] + mydata['hitop236'] + mydata['hitop258']

# Convergent validity hypotheses

In [47]:
# The p-value helps us determine whether or not we can meaningfully conclude that 
# the population correlation coefficient is different from zero, 
# based on what we observe from the sample.

In [50]:
def do_correlation(col1, col2):
    corr_result = pg.corr(col1, col2, method = 'spearman')
    my_r = corr_result.iloc[0]['r']
    my_r = "{:.2f}".format(float(my_r))
    my_ci = corr_result.iloc[0]['CI95%']
    my_ci_1 = my_ci[0]
    my_ci_2 = my_ci[1]
    my_p = corr_result.iloc[0]['p-val']
    my_p = "{:.3f}".format(float(my_p)) 
    return(my_p, my_r, my_ci_1, my_ci_2)

def check_convergent_hypotheses(visit, whichdata):
    assert(visit in ['initial'])
    assert(whichdata in ['genpop', 'enriched', 'combined'])
    print('\nDATA = '+whichdata)    
    if visit == 'initial':
        print('\nVISIT = '+visit)
        # HYPOTHESIS C1: hitop_anhedonic_depression will be positively correlated with PHQ-8
        # Spearman; alpha = .05
        print('\nHYPOTHESIS C1')
        if whichdata == 'genpop':
            (p, r, ci1, ci2) = do_correlation(mydata.hitop_anhedonic_depression, mydata.phq_sum)
        elif whichdata == 'enriched':
            (p, r, ci1, ci2) = do_correlation(mydata.hitop_anhedonic_depression_invcore, mydata.phq_sum) 
        elif whichdata == 'combined':
            (p, r, ci1, ci2) = do_correlation(mydata.hitop_anhedonic_depression_invcore, mydata.phq_sum)
        if float(p) >= 0.05:
            print(f"Correlation NOT Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
        else:
            print(f"Correlation Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
            if float(r) > 0:
                print('Hyp C1 SUPPORTED: Anhedonic depression (or invariant core for enriched) positively correlates with PHQ-8')
            else:
                print('Hyp C1 NOT SUPPORTED: Anhedonic depression (or invariant core for enriched) DOES NOT positively correlate with PHQ-8')
        
        # HYPOTHESIS C2: hitop_well-being will be negatively correlated with PHQ-8
        # Spearman; alpha = .05
        print('\nHYPOTHESIS C2') 
        if whichdata == 'genpop':
            (p, r, ci1, ci2) = do_correlation(mydata.hitop_well_being_invcore, mydata.phq_sum)
        elif whichdata == 'enriched':
            (p, r, ci1, ci2) = do_correlation(mydata.hitop_well_being, mydata.phq_sum)      
        elif whichdata == 'combined':
            (p, r, ci1, ci2) = do_correlation(mydata.hitop_well_being, mydata.phq_sum)
        if float(p) >= 0.05:
            print(f"Correlation NOT Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
        else:
            print(f"Correlation Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
            if float(r) < 0:
                print('Hyp C2 SUPPORTED: Well-being (or invariant core for genpop) negatively correlates with PHQ-8')
            else:
                print('Hyp C2 NOT SUPPORTED: Well-being (or invariant core for genpop) DOES NOT negatively correlate with PHQ-8')
        
        # HYPOTHESIS C3: hitop_anxious_worry will be positively correlated with GAD-7
        # Spearman; alpha = .05
        print('\nHYPOTHESIS C3')            
        if whichdata == 'genpop':
            (p, r, ci1, ci2) = do_correlation(mydata.hitop_anxious_worry_invcore, mydata.gad_sum)
        elif whichdata == 'enriched':   
            (p, r, ci1, ci2) = do_correlation(mydata.hitop_anxious_worry, mydata.gad_sum)
        elif whichdata == 'combined':   
            (p, r, ci1, ci2) = do_correlation(mydata.hitop_anxious_worry_invcore, mydata.gad_sum)
        if float(p) >= 0.05:
            print(f"Correlation NOT Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
        else:
            print(f"Correlation Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
            if float(r) > 0:
                print('Hyp C3 SUPPORTED: Anxious worry (or invariant core for genpop) positively correlates with GAD-7')
            else:
                print('Hyp C3 NOT SUPPORTED: Anxious worry (or invariant core for genpop) DOES NOT positively correlate with GAD-7') 

        # HYPOTHESIS C4: Higher sum of scores on the HiTOP Appetite Gain and Appetite Loss 
        # will be correlated with responses to PJQ-8 "Poor appetite or overeating"
        print('\nHYPOTHESIS C4')            
        if whichdata == 'genpop':
            mydata['hitop_appetite_sum'] = mydata['hitop_appetite_gain_invcore'] + mydata['hitop_appetite_loss']
        elif whichdata == 'enriched':   
            mydata['hitop_appetite_sum'] = mydata['hitop_appetite_gain'] + mydata['hitop_appetite_loss']
        elif whichdata == 'combined':   
            mydata['hitop_appetite_sum'] = mydata['hitop_appetite_gain'] + mydata['hitop_appetite_loss']
        (p, r, ci1, ci2) = do_correlation(mydata.hitop_appetite_sum, mydata.phq_5)
        if float(p) >= 0.05:
            print(f"Correlation NOT Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
            print('Hyp C4 NOT SUPPORTED: Sum of scores on Appetite Gain and Appetite Loss (or invariant cores) DOES NOT correlate with PHQ item 5 - poor appetite or overeating')         
        else:
            print(f"Correlation Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
            print('Hyp C4 SUPPORTED: Sum of scores on Appetite Gain and Appetite Loss (or invariant cores) is correlated with PHQ item 5 - poor appetite or overeating')

        # HYPOTHESIS C5: the HiTOP Cognitive Problems will be positively correlated with responses 
        # to the PHQ-8 item "Trouble concentrating on things, such as school work, reading or watching television".
        print('\nHYPOTHESIS C5')            
        (p, r, ci1, ci2) = do_correlation(mydata.hitop_cognitive_problems, mydata.phq_7)
        if float(p) >= 0.05:
            print(f"Correlation NOT Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
        else:
            print(f"Correlation Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
            if float(r) > 0:
                print('Hyp C5 SUPPORTED: HiTOP Cognitive problems positive correlates with PHQ-8 item 7 "Trouble concentrating on things..."')
            else:
                print('Hyp C5 NOT SUPPORTED: HiTOP Cognitive problems DOES NOT positive correlate with PHQ-8 item 7 "Trouble concentrating on things..."')         
        
        # HYPOTHESIS C6: The HiTOP Insomnia subscale will be positively correlated with responses 
        # to the PHQ-8 item "Trouble falling or staying asleep, or sleeping too much"
        print('\nHYPOTHESIS C6')            
        (p, r, ci1, ci2) = do_correlation(mydata.hitop_insomnia, mydata.phq_3)        
        if float(p) >= 0.05:
            print(f"Correlation NOT Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
        else:
            print(f"Correlation Significant with p-val = {p} and r = {r}({ci1}, {ci2})")
            if float(r) > 0:
                print('Hyp C6 SUPPORTED: HiTOP Insomnia positive correlates with PHQ-8 item 3 "Trouble falling or staying asleep, or sleeping too much"')
            else:
                print('Hyp C6 NOT SUPPORTED: HiTOP Insomnia DOES NOT positive correlate with PHQ-8 item 3 "Trouble falling or staying asleep, or sleeping too much"')         

        # HYPOTHESIS C7: Higher sum of all HiTOP subscales will be associated with greater likelihood
        # of being bothered by symptoms of a diagnosed mood or anxiety disorder. 
        print('\nHYPOTHESIS C7')            
        if whichdata == 'genpop':
            mydata['hitop_sum_new'] = mydata['hitop_anhedonic_depression'] + mydata['hitop_anxious_worry_invcore'] + \
            mydata['hitop_appetite_gain_invcore'] + mydata['hitop_appetite_loss'] + mydata['hitop_cognitive_problems'] + \
            mydata['hitop_hyposomnia'] + mydata['hitop_indecisiveness'] + mydata['hitop_insomnia'] + mydata['hitop_panic'] + \
            mydata['hitop_separation_insecurity_invcore'] + mydata['hitop_shame_guilt'] + mydata['hitop_situational_phobia'] + \
            mydata['hitop_social_anxiety_invcore']
        elif whichdata == 'enriched':   
            mydata['hitop_sum_new'] = mydata['hitop_anhedonic_depression_invcore'] + mydata['hitop_anxious_worry'] + \
            mydata['hitop_appetite_gain'] + mydata['hitop_appetite_loss'] + mydata['hitop_cognitive_problems'] + \
            mydata['hitop_hyposomnia'] + mydata['hitop_indecisiveness'] + mydata['hitop_insomnia'] + mydata['hitop_panic'] + \
            mydata['hitop_separation_insecurity_invcore'] + mydata['hitop_shame_guilt'] + mydata['hitop_situational_phobia'] + \
            mydata['hitop_social_anxiety_invcore']
        elif whichdata == 'combined':   
            mydata['hitop_sum_new'] = mydata['hitop_anhedonic_depression_invcore'] + mydata['hitop_anxious_worry_invcore'] + \
            mydata['hitop_appetite_gain'] + mydata['hitop_appetite_loss'] + mydata['hitop_cognitive_problems'] + \
            mydata['hitop_hyposomnia'] + mydata['hitop_indecisiveness'] + mydata['hitop_insomnia'] + mydata['hitop_panic'] + \
            mydata['hitop_separation_insecurity'] + mydata['hitop_shame_guilt'] + mydata['hitop_situational_phobia'] + \
            mydata['hitop_social_anxiety_invcore']
        # using an example from here: https://www.science.smith.edu/~jcrouser/SDS293/labs/lab4-py.html
        formula = 'moodanxiety_bothered ~ hitop_sum_new'        
        model = smf.glm(formula = formula, data=mydata, family=sm.families.Binomial())
        result = model.fit()
        print(result.summary())
        p_val_hitop_sum = format(result.pvalues.iloc[1], 'f')
        if float(p_val_hitop_sum) >= 0.05:
            print(f"\nHitop sum of items is NOT a significant predictor of mood_anxiety_bothered, with p-val = {p_val_hitop_sum}")
            print('Hyp C7 NOT SUPPORTED')         
        else:
            print(f"\nHitop sum of items IS A SIGNIFICANT PREDICTOR of mood_anxiety_bothered, with p-val = {p_val_hitop_sum}")
            print('Hyp C7 SUPPORTED')

In [51]:
check_convergent_hypotheses(visit = 'initial', whichdata = 'combined')


DATA = combined

VISIT = initial

HYPOTHESIS C1
Correlation Significant with p-val = 0.000 and r = 0.87(0.85, 0.89)
Hyp C1 SUPPORTED: Anhedonic depression (or invariant core for enriched) positively correlates with PHQ-8

HYPOTHESIS C2
Correlation Significant with p-val = 0.000 and r = -0.55(-0.6, -0.5)
Hyp C2 SUPPORTED: Well-being (or invariant core for genpop) negatively correlates with PHQ-8

HYPOTHESIS C3
Correlation Significant with p-val = 0.000 and r = 0.89(0.87, 0.9)
Hyp C3 SUPPORTED: Anxious worry (or invariant core for genpop) positively correlates with GAD-7

HYPOTHESIS C4
Correlation Significant with p-val = 0.000 and r = 0.77(0.73, 0.8)
Hyp C4 SUPPORTED: Sum of scores on Appetite Gain and Appetite Loss (or invariant cores) is correlated with PHQ item 5 - poor appetite or overeating

HYPOTHESIS C5
Correlation Significant with p-val = 0.000 and r = 0.82(0.79, 0.85)
Hyp C5 SUPPORTED: HiTOP Cognitive problems positive correlates with PHQ-8 item 7 "Trouble concentrating on thi

# Divergent validity hypotheses

In [55]:
for c in mydata.columns:
    print(c)

Subject
hitop_sum
hitop_sum_recontact
baars_sum
baars_sum_recontact
phq_sum
phq_sum_recontact
gad_sum
gad_sum_recontact
mood_bothered
mood_bothered_recontact
anxiety_bothered
anxiety_bothered_recontact
attention_bothered
attention_bothered_recontact
moodanxiety_bothered
moodanxiety_bothered_recontact
baars_inattention_sum
baars_inattention_sum_recontact
baars_hyperactivity_sum
baars_hyperactivity_sum_recontact
baars_impulsivity_sum
baars_impulsivity_sum_recontact
baars_sct_sum
baars_sct_sum_recontact
hitop_anhedonic_depression
hitop_anhedonic_depression_recontact
hitop_anxious_worry
hitop_anxious_worry_recontact
hitop_appetite_gain
hitop_appetite_gain_recontact
hitop_appetite_loss
hitop_appetite_loss_recontact
hitop_cognitive_problems
hitop_cognitive_problems_recontact
hitop_hyposomnia
hitop_hyposomnia_recontact
hitop_indecisiveness
hitop_indecisiveness_recontact
hitop_insomnia
hitop_insomnia_recontact
hitop_panic
hitop_panic_recontact
hitop_separation_insecurity
hitop_separation_insec

In [106]:
whichdata = 'combined'
visit = 'initial'

print('\nDATA = ' + whichdata)    
print('\nVISIT = ' + visit)

# PREP FOR HYP D1 AND HYP D2

# -----------------------------------------------------------------------
# D1: There will be a stronger correlation between the sum of HiTOP scales and the sum of GAD-7 and PHQ-8 scores 
#     than with BAARS-IV total score

mydata['gad_phq_sum'] = mydata['gad_sum'] + mydata['phq_sum']
mydata['hitop_sum_invcores'] = mydata['hitop_anhedonic_depression_invcore'] + mydata['hitop_anxious_worry_invcore'] + \
            mydata['hitop_appetite_gain'] + mydata['hitop_appetite_loss'] + mydata['hitop_cognitive_problems'] + \
            mydata['hitop_hyposomnia'] + mydata['hitop_indecisiveness'] + mydata['hitop_insomnia'] + mydata['hitop_panic'] + \
            mydata['hitop_separation_insecurity'] + mydata['hitop_shame_guilt'] + mydata['hitop_situational_phobia'] + \
            mydata['hitop_social_anxiety_invcore']
(p_hitop_gadphq, r_hitop_gadphq, ci1_hitop_gadphq, ci2_hitop_gadphq) = do_correlation(stats.zscore(mydata.hitop_sum_invcores), stats.zscore(mydata.gad_phq_sum))
(p_hitop_baars, r_hitop_baars, ci1_hitop_baars, ci2_hitop_baars) = do_correlation(stats.zscore(mydata.hitop_sum_invcores), stats.zscore(mydata.baars_sum))
corr_result_orig = float(r_hitop_gadphq) - float(r_hitop_baars)

# -----------------------------------------------------------------------
# D2: The sum of HiTOP scales will be a better predictor of being bothered by symptoms of a diagnosed anxiety disorder 
# than of being bothered by symptoms of ADHD.

X_hitop_sum_invcores = []
for x in mydata['hitop_sum_invcores'].tolist():
    X_hitop_sum_invcores.append([x])
    
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html
mi_hitop_moodanxboth = mutual_info_classif(X_hitop_sum_invcores, mydata['moodanxiety_bothered'].tolist())
mi_hitop_attnboth = mutual_info_classif(X_hitop_sum_invcores, mydata['attention_bothered'].tolist())
MI_diff_orig_D2 = mi_hitop_moodanxboth - mi_hitop_attnboth

# -----------------------------------------------------------------------
# D3: The BAARS-IV total score will be a better predictor of being bothered by symptoms of ADHD 
# than of a diagnosed mood or anxiety disorder.

X_baars_sum = []
for x in mydata['baars_sum'].tolist():
    X_baars_sum.append([x])
    
mi_baars_attnboth = mutual_info_classif(X_baars_sum, mydata['attention_bothered'].tolist())
mi_baars_moodanxboth = mutual_info_classif(X_baars_sum, mydata['moodanxiety_bothered'].tolist())
MI_diff_orig_D3 = mi_baars_attnboth - mi_baars_moodanxboth

# -----------------------------------------------------------------------
# D4: The sum of HiTOP mood disorder scales will be a better predictor of being bothered by symptoms 
# of a diagnosed mood disorder than of being bothered by symptoms of a diagnosed anxiety disorder.

X_hitop_depression = []
for x in mydata['hitop_anhedonic_depression_invcore'].tolist():
    X_hitop_depression.append([x])

mi_hitopdepr_moodboth = mutual_info_classif(X_hitop_depression, mydata['mood_bothered'].tolist())
mi_hitopdepr_anxboth = mutual_info_classif(X_hitop_depression, mydata['anxiety_bothered'].tolist())
MI_diff_orig_D4 = mi_hitopdepr_moodboth - mi_hitopdepr_anxboth

# -----------------------------------------------------------------------
# D5: The sum of HiTOP anxiety disorder scales will be a better predictor of being bothered by symptoms 
# of a diagnosed anxiety disorder than of being bothered by symptoms of a diagnosed mood disorder.

X_hitop_anxiety = []
for x in mydata['hitop_anxious_worry_invcore'].tolist():
    X_hitop_anxiety.append([x])
    
mi_hitopanxiety_anxboth = mutual_info_classif(X_hitop_anxiety, mydata['anxiety_bothered'].tolist())
mi_hitopanxiety_moodboth = mutual_info_classif(X_hitop_anxiety, mydata['mood_bothered'].tolist())
MI_diff_orig_D5 = mi_hitopanxiety_anxboth - mi_hitopanxiety_moodboth

# -----------------------------------------------------------------------
corr_result_perm = []
MI_diff_perm_D2 = []
MI_diff_perm_D3 = []
MI_diff_perm_D4 = []
MI_diff_perm_D5 = []

# permutations
for permut in range (1000):
    rand_seed = randint(1, 10000)
    dat_shuffled = pd.DataFrame()
    dat_shuffled['hitop_sum_invcores'] = mydata['hitop_sum_invcores']
    np.random.seed(rand_seed)
    dat_shuffled['baars_sum'] = np.random.permutation(mydata['baars_sum'])
    np.random.seed(rand_seed)
    dat_shuffled['gad_phq_sum'] = np.random.permutation(mydata['gad_phq_sum'])
    np.random.seed(rand_seed)
    dat_shuffled['moodanxiety_bothered'] = np.random.permutation(mydata['moodanxiety_bothered'])
    np.random.seed(rand_seed)
    dat_shuffled['attention_bothered'] = np.random.permutation(mydata['attention_bothered'])
    np.random.seed(rand_seed)
    dat_shuffled['mood_bothered'] = np.random.permutation(mydata['mood_bothered'])
    np.random.seed(rand_seed)
    dat_shuffled['anxiety_bothered'] = np.random.permutation(mydata['anxiety_bothered'])
    # D1
    (p_hitop_gadphq_curr, r_hitop_gadphq_curr, ci1_hitop_gadphq_curr, ci2_hitop_gadphq_curr) = do_correlation(stats.zscore(dat_shuffled.hitop_sum_invcores), stats.zscore(dat_shuffled.gad_phq_sum))
    (p_hitop_baars_curr, r_hitop_baars_curr, ci1_hitop_baars_curr, ci2_hitop_baars_curr) = do_correlation(stats.zscore(dat_shuffled.hitop_sum_invcores), stats.zscore(dat_shuffled.baars_sum))
    corr_result_perm.append(float(r_hitop_gadphq_curr) - float(r_hitop_baars_curr))
    # D2
    mi_hitop_moodanxboth_perm = mutual_info_classif(X_hitop_sum_invcores, dat_shuffled['moodanxiety_bothered'].tolist())
    mi_hitop_attnboth_perm = mutual_info_classif(X_hitop_sum_invcores, dat_shuffled['attention_bothered'].tolist())   
    MI_diff_perm_D2.append(mi_hitop_moodanxboth_perm - mi_hitop_attnboth_perm)
    # D3
    mi_baars_attnboth_perm = mutual_info_classif(X_baars_sum, dat_shuffled['attention_bothered'].tolist())
    mi_baars_moodanxboth_perm = mutual_info_classif(X_baars_sum, dat_shuffled['moodanxiety_bothered'].tolist())
    MI_diff_perm_D3.append(mi_baars_attnboth_perm - mi_baars_moodanxboth_perm)
    # D4
    mi_hitopdepr_moodboth_perm = mutual_info_classif(X_hitop_depression, dat_shuffled['mood_bothered'].tolist())
    mi_hitopdepr_anxboth_perm = mutual_info_classif(X_hitop_depression, dat_shuffled['anxiety_bothered'].tolist())
    MI_diff_perm_D4.append(mi_hitopdepr_moodboth_perm - mi_hitopdepr_anxboth_perm)    
    # D5
    mi_hitopanxiety_anxboth_perm = mutual_info_classif(X_hitop_anxiety, dat_shuffled['anxiety_bothered'].tolist())
    mi_hitopanxiety_moodboth_perm = mutual_info_classif(X_hitop_anxiety, dat_shuffled['mood_bothered'].tolist())
    MI_diff_perm_D5.append(mi_hitopanxiety_anxboth_perm - mi_hitopanxiety_moodboth_perm)

# -----------------------------------------------------------------------
# D1: There will be a stronger correlation between the sum of HiTOP scales and the sum of GAD-7 and PHQ-8 scores 
#     than with BAARS-IV total score
print('\nHYPOTHESIS D1')

# Perform one-sample t-test
t_stat, p_value = stats.ttest_1samp(corr_result_perm, corr_result_orig)
print("\nT statistic:", t_stat)
print("P-value:", p_value)

# Setting significance level
alpha = 0.05

# Interpret the results
if p_value < alpha:
    print("\nHyp D1: Reject the null hypothesis; there is a significant difference between the permutation mean and the actual mean.")
else:
    print("\nHyp D1: Fail to reject the null hypothesis; there is no significant difference between the permutation mean and the actual mean.")

# -----------------------------------------------------------------------
# D2: The sum of HiTOP scales will be a better predictor of being bothered by symptoms of a diagnosed anxiety disorder 
# than of being bothered by symptoms of ADHD.
print('\nHYPOTHESIS D2')

print('\nActual MI between hitop and mood_anx_bothered: ' + str(mi_hitop_moodanxboth))
print('Actual MI between hitop and attnt_bothered: ' + str(mi_hitop_attnboth))

# Perform one-sample t-test
t_stat, p_value = stats.ttest_1samp(MI_diff_perm_D2, MI_diff_orig_D2)
print("\nT statistic:", t_stat)
print("P-value:", p_value)

# Setting significance level
alpha = 0.05

# Interpret the results
if p_value < alpha:
    print("\nHyp D2: Reject the null hypothesis; there is a significant difference between the permutation mean and the actual mean.")
else:
    print("\nHyp D2: Fail to reject the null hypothesis; there is no significant difference between the permutation mean and the actual mean.")

# -----------------------------------------------------------------------
# D3: The BAARS-IV total score will be a better predictor of being bothered by symptoms of ADHD 
# than of a diagnosed mood or anxiety disorder.
print('\nHYPOTHESIS D3')

print('\nActual MI between baars and mood_anx_bothered: ' + str(mi_baars_moodanxboth))
print('Actual MI between baars and attnt_bothered: ' + str(mi_baars_attnboth))

# Perform one-sample t-test
t_stat, p_value = stats.ttest_1samp(MI_diff_perm_D3, MI_diff_orig_D3)
print("\nT statistic:", t_stat)
print("P-value:", p_value)

# Setting significance level
alpha = 0.05

# Interpret the results
if p_value < alpha:
    print("\nHyp D3: Reject the null hypothesis; there is a significant difference between the permutation mean and the actual mean.")
else:
    print("\nHyp D3: Fail to reject the null hypothesis; there is no significant difference between the permutation mean and the actual mean.")

# -----------------------------------------------------------------------
# D4: The sum of HiTOP mood disorder scales will be a better predictor of being bothered by symptoms 
# of a diagnosed mood disorder than of being bothered by symptoms of a diagnosed anxiety disorder.
print('\nHYPOTHESIS D4:')

print('\nActual MI between HiTOP depresson scale and and mood_bothered: ' + str(mi_hitopdepr_moodboth))
print('Actual MI between HiTOP depression scale and anxiety_bothered: ' + str(mi_hitopdepr_anxboth))

# Perform one-sample t-test
t_stat, p_value = stats.ttest_1samp(MI_diff_perm_D4, MI_diff_orig_D4)
print("\nT statistic:", t_stat)
print("P-value:", p_value)

# Setting significance level
alpha = 0.05

# Interpret the results
if p_value < alpha:
    print("\nHyp D4: Reject the null hypothesis; there is a significant difference between the permutation mean and the actual mean.")
else:
    print("\nHyp D4: Fail to reject the null hypothesis; there is no significant difference between the permutation mean and the actual mean.")

# -----------------------------------------------------------------------
# D5: The sum of HiTOP anxiety disorder scales will be a better predictor of being bothered by symptoms 
# of a diagnosed anxiety disorder than of being bothered by symptoms of a diagnosed mood disorder.
print('\nHYPOTHESIS D5')

X_hitop_anxiety = []
for x in mydata['hitop_anxious_worry_invcore'].tolist():
    X_hitop_anxiety.append([x])
    
mi_hitopanxiety_anxboth = mutual_info_classif(X_hitop_anxiety, mydata['anxiety_bothered'].tolist())
mi_hitopanxiety_moodboth = mutual_info_classif(X_hitop_anxiety, mydata['mood_bothered'].tolist())
MI_diff_orig_D5 = mi_hitopanxiety_anxboth - mi_hitopanxiety_moodboth

print('\nActual MI between HiTOP anxiety scale and and anxiety_bothered: ' + str(mi_hitopanxiety_anxboth))
print('Actual MI between HiTOP anxiety scale and mood_bothered: ' + str(mi_hitopanxiety_moodboth))

# Perform one-sample t-test
t_stat, p_value = stats.ttest_1samp(MI_diff_perm_D5, MI_diff_orig_D5)
print("\nT statistic:", t_stat)
print("P-value:", p_value)

# Setting significance level
alpha = 0.05

# Interpret the results
if p_value < alpha:
    print("\nHyp D5: Reject the null hypothesis; there is a significant difference between the permutation mean and the actual mean.")
else:
    print("\nHyp D5: Fail to reject the null hypothesis; there is no significant difference between the permutation mean and the actual mean.")



DATA = combined

VISIT = initial

HYPOTHESIS D1

T statistic: -130.50532484781212
P-value: 0.0

Hyp D1: Reject the null hypothesis; there is a significant difference between the permutation mean and the actual mean.

HYPOTHESIS D2

Actual MI between hitop and mood_anx_bothered: [0.26902607]
Actual MI between hitop and attnt_bothered: [0.06686658]

T statistic: [-417.83128938]
P-value: [0.]

Hyp D2: Reject the null hypothesis; there is a significant difference between the permutation mean and the actual mean.

HYPOTHESIS D3

Actual MI between baars and mood_anx_bothered: [0.20029271]
Actual MI between baars and attnt_bothered: [0.12335384]

T statistic: [165.16827886]
P-value: [0.]

Hyp D3: Reject the null hypothesis; there is a significant difference between the permutation mean and the actual mean.

HYPOTHESIS D4:

Actual MI between HiTOP depresson scale and and mood_bothered: [0.18000076]
Actual MI between HiTOP depression scale and anxiety_bothered: [0.15201509]

T statistic: [-57.